# Notebook to form interferograms using ESA SNAP. Shifting to SNAP because of possible NASA funding cuts; might impact the support for both ISCE2 and ISCE3

In [ ]:
import os
import subprocess
from datetime import datetime
import shutil
from osgeo import gdal
import numpy as np
import netCDF4 as nc
import xarray as xr
import matplotlib.pyplot as plt
import itertools

# Functions

In [ ]:
def gpt_help():

    cmd = f'{GPT_PATH} -h -c {MEMORY_SIZE}'

    try:
        result = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

def help_cmd(cmdname):

    cmd = f'{GPT_PATH} {cmdname} -h -c {MEMORY_SIZE}'

    try:
        result = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

def project_dir(work_dir):
    """
    This function reads in a string that you wish to make your working directory 
    for the InSAR project, and creates a data directory to store the data for ISCE2 and mintpy
    work-dir = str
        path to the directory created in 01_get_slc.ipynb
    """

    #creates file on your desktop containing the work of this notebook
    os.makedirs(work_dir, exist_ok=True, mode=0o777)
    
    # file inside work_dir for isce2 interferometry
    if_dir = os.path.join(work_dir,'InSAR/interferometry')
    os.makedirs(if_dir, exist_ok=True, mode=0o777)
    
    # file inside work_dir for mintpy time-series
    ts_dir = os.path.join(work_dir,'InSAR/time_series')
    os.makedirs(ts_dir, exist_ok=True, mode=0o777)

    return if_dir, ts_dir

# Establish GPT path, number of processors, and memory usage

In [ ]:
GPT_PATH = '/home/clay/esa-snap/bin/gpt'          #linux
# GPT_PATH = '/Applications/esa-snap/bin/gpt'         #mac
NUM_PROCESSORS = 12                                 # 24 total on my linux, 14 total on mac
MEMORY_SIZE = '40G'                                 # 96G total on my linux, 24G total on mac
SNAPHU_PATH = '/usr/local/bin/snaphu'               # path to snaphu for unwrapping

# Get SLC images that were downloaded

In [ ]:
# assuming you have downloaded .zip files covering your AOI from ASF Vertex
# enter the file directory below
slc_zips = '/home/clay/Documents/SabineRS/Sentinel-1/SLC/ASCENDING/136/93'

slc_zips_list = sorted(os.listdir(slc_zips), key=lambda x: datetime.strptime(x[17:25], '%Y%m%d'))
slc_zips_dirs = [os.path.join(slc_zips, slc) for slc in slc_zips_list]
slc_zips_dates = [slc[17:25] for slc in slc_zips_list]

# Establish working directories for SNAP interferogram and MintPy
Needs for MintPy are:
1. Wrapped Ifg
2. Elevation Band
3. Coherence Band
4. Unwrapped ifg

Directory structure much simpler than ISCE:
1. Ifg directory
    - stores wrapped ifg .dim and .data files (optional) 
    - stores coherence band .dim and .data files
    - stores elevation band .dim and .data files
    - stores unwrapped ifg .dim and .data files
2. Referance DEM .dim and .data file
3. MintPy directory
    - MintPy .txt config file?

In [ ]:
proj_dir = '/home/clay/Documents/SabineRS'
work_dir = os.path.join(proj_dir, 'Sentinel-1')
if_dir, ts_dir= project_dir(work_dir)

# Choose which polarizations and DEM you want to use for processing

In [ ]:
# can define this here for the rest of the notebook
# typically VV has highest coherence due to reliance on surface scattering
# VH largely impacted by volumetric scattering present in vegetated areas

POLARISATIONS = 'VV' 

# repeated a lot, so leaving this here as well
DEM_NAME = 'Copernicus 30m Global DEM'

# 1. TOPSAR-Split
- will try to add something in here where can input an aoi and automatically identify the swaths needed
- wkt aoi can be passed to get the bursts, but not sure if it also works for subswaths
- ABraun recommends splitting before applying orbit file, maybe will save time?
- do for all images

- May need to just go in manually for the reference file to identify subswaths and bursts :(

In [ ]:
# for this, could be interesting to keep both polarisations
# would use VV for the interferograms but could include VH for coherence time-series?
# this uses all subswaths. Will probably take longer but it make

#after manual inspection
# maybe add a for loop to do more than one subswath (e.g. IW2 and IW3)
SUBSWATH = 'IW3'
FIRSTBURST = 2
LASTBURST = 6

# with open('/Users/clayc/Documents/Dissertation/SabineRS/wkt_aoi.txt') as f:        
#     lines = f.readlines()
# aoi = lines[0]

splits_path = os.path.join(if_dir, '01_split')
os.makedirs(splits_path, exist_ok=True, mode=0o777)

split_outpaths = []
for i, file in enumerate(slc_zips_list):
    split_outpaths.append(os.path.join(splits_path, f'{slc_zips_dates[i]}.dim'))
    split_cmd = f'{GPT_PATH} TOPSAR-Split -Ssource={os.path.join(slc_zips,file)} -PselectedPolarisations={POLARISATIONS} -Psubswath="{SUBSWATH}" -PfirstBurstIndex={FIRSTBURST} -PlastBurstIndex={LASTBURST} -t {split_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(split_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

# 2. Apply Orbit File

In [ ]:
ORBIT_TYPE = 'Sentinel Precise (Auto Download)'     # str, options include'Sentinel Precise (Auto Download)', 'Sentinel Restituted (Auto Download)', 'DORIS Preliminary POR (ENVISAT)', 'DORIS Precise VOR (ENVISAT) (Auto Download)', 'DELFT Precise (ENVISAT, ERS1&2) (Auto Download)', 'PRARE Precise (ERS1&2) (Auto Download)', 'Kompsat5 Precise'
ORBIT_DEGREE = 3                                     # int
CONTINUE_ON_FAIL = False

orbits_path = os.path.join(if_dir, '02_orbit')
os.makedirs(orbits_path, exist_ok=True, mode=0o777)

orbit_outpaths = []
for i, file in enumerate(slc_zips_list):
    orbit_outpaths.append(os.path.join(orbits_path, f'{slc_zips_dates[i]}.dim'))
    orbit_cmd = f'{GPT_PATH} Apply-Orbit-File -Ssource={split_outpaths[i]} -PcontinueOnFail={CONTINUE_ON_FAIL} -PorbitType="{ORBIT_TYPE}" -PpolyDegree={ORBIT_DEGREE} -t {orbit_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(orbit_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(splits_path)

# 3. Back-geocoding Co-reg
- this one is funky. Need to pass the two files for the ifg together?
- pass with -SsourceProducts?
- look into this more, may need to pass the two SLCs being use to produce each ifg

In [ ]:
orbits_path = os.path.join(if_dir, '02_orbit')
os.makedirs(orbits_path, exist_ok=True, mode=0o777)

orbit_outpaths = []
for i, file in enumerate(slc_zips_list):
    orbit_outpaths.append(os.path.join(orbits_path, f'{slc_zips_dates[i]}.dim'))

pairs = []
# Generate triplets for every consecutive set of 3 images
for i in range(len(orbit_outpaths)-2):  
    # Form the triplet from 3 consecutive images
    A = orbit_outpaths[i]
    B = orbit_outpaths[i+1]
    C = orbit_outpaths[i+2]
    
    # Create the three pairs from each triplet
    pairs.append((A,B))  # i and i+1
    pairs.append((B,C))  # i+1 and i+2 
    pairs.append((C,A))  # i+2 and i

# Remove any duplicate pairs
pairs = list(set(pairs))

In [ ]:
for i, pair in enumerate(pairs[:3]):
    filename = f'{pairs[i][0][-12:-4]}_{pairs[i][1][-12:-4]}.dim'
    pair_str=f'{pairs[i][0]},{pairs[i][1]}'
    print(filename)
    print(pair_str)

In [ ]:
# ### EXAMPLE .XML FILE FOR back geocoding
# format similar to the nc_export.xml I created

# <graph id="Graph">
#   <version>1.0</version>
#   <node id="Back-Geocoding">
#     <operator>Back-Geocoding</operator>
#     <sources>
#       <sourceProduct.2 refid="ProductSet-Reader"/>
#     </sources>
#     <parameters class="com.bc.ceres.binding.dom.XppDomElement">
#       <demName>${demName}</demName>
#       <demResamplingMethod>${demResamplingMethod}</demResamplingMethod>
#       <externalDEMFile/>
#       <externalDEMNoDataValue>0.0</externalDEMNoDataValue>
#       <resamplingType>${resamplingType}</resamplingType>
#       <maskOutAreaWithoutElevation>${maskOutAreaWithoutElevation}</maskOutAreaWithoutElevation>
#       <outputRangeAzimuthOffset>${outputRangeAzimuthOffset}</outputRangeAzimuthOffset>
#       <outputDerampDemodPhase>${outputDerampDemodPhase}</outputDerampDemodPhase>
#       <disableReramp>${disableReramp}</disableReramp>
#     </parameters>
#   </node>
#   <node id="Write">
#     <operator>Write</operator>
#     <sources>
#       <sourceProduct refid="Back-Geocoding"/>
#     </sources>
#     <parameters class="com.bc.ceres.binding.dom.XppDomElement">
#       <file>${output}</file>
#       <formatName>BEAM-DIMAP</formatName>
#     </parameters>
#   </node>
#   <node id="ProductSet-Reader">
#     <operator>ProductSet-Reader</operator>
#     <sources/>
#     <parameters class="com.bc.ceres.binding.dom.XppDomElement">
#       <fileList>${fileList}</fileList>
#     </parameters>
#   </node>
# </graph>

In [ ]:
DEM_RESAMPLE_METHOD = 'BICUBIC_INTERPOLATION'       # str, options include 'NEAREST_NEIGHBOUR', 'BILINEAR_INTERPOLATION', 'CUBIC_CONVOLUTION', 'BISINC_5_POINT_INTERPOLATION', 'BISINC_11_POINT_INTERPOLATION', 'BISINC_21_POINT_INTERPOLATION', 'BICUBIC_INTERPOLATION'
RESAMPLE_METHOD = 'BISINC_5_POINT_INTERPOLATION'    # str, options inlcude 'NEAREST_NEIGHBOUR', 'BILINEAR_INTERPOLATION', 'CUBIC_CONVOLUTION', 'BISINC_5_POINT_INTERPOLATION', 'BISINC_11_POINT_INTERPOLATION', 'BISINC_21_POINT_INTERPOLATION', 'BICUBIC_INTERPOLATION'

geocodes_path = os.path.join(if_dir, '03_geocode')
os.makedirs(geocodes_path, exist_ok=True, mode=0o777)

geocode_outpaths = []
for i, pair in enumerate(pairs):
    geocode_outpaths.append(os.path.join(geocodes_path, f'{pairs[i][0][-12:-4]}_{pairs[i][1][-12:-4]}.dim'))
    ref = pairs[i][0]
    sec = pairs[i][1]
    # pair_str=f'{pairs[i][0]},{pairs[i][1]}'

    geocode_cmd = f'{GPT_PATH} Back-Geocoding -SsourceProducts="{ref}" "{sec}" -PdemName="{DEM_NAME}" -PdemResamplingMethod="{DEM_RESAMPLE_METHOD}" -PresamplingType={RESAMPLE_METHOD} -t {geocode_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(geocode_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

# shutil.rmtree(orbits_path)

# 4a. ESD (only if using more than one burst. which is pretty typical)

In [ ]:
COHERENCE_THRESHOLD = 0.3           # (0, 1]
ESD_ESTIMATOR = 'Periodogram'       # Average or Periodogram

#documentation not very good, assuming this is just resampling done before estimating ESD?
# will look at STEP forum for more info
WIN_ACC_AZI = 16                    # 2, 4, 8, 16, 32, 64
WIN_ACC_RANGE = 16                  # 2, 4, 8, 16, 32, 64
WIN_HEIGHT = 512                    # 32, 64, 128, 256, 512, 1024, 2048
WIN_OVERSAMPLING = 128              # 32, 64, 128, 256
WIN_WIDTH = 512                     # 32, 64, 128, 256, 512, 1024, 2048

INTEGRATION_METHOD = 'L1 and L2'    # L1, L2, L1 and L2
NUM_BLOCKS_PER_OVERLAP = 10         # [1,20]
OVERALL_AZI_SHIFT = 0.0             
OVERALL_RANGE_SHIFT = 0.0           
WEIGHT_FUNCTION = 'Inv Quadratic'   # None, Linear, Quadratic, Inv Quadratic
XCOR_THRESHOLD = 0.1                #(0, *)

esds_path = os.path.join(if_dir, '04_ESD')
os.makedirs(esds_path, exist_ok=True, mode=0o777)

esd_outpaths = []
for i, file in enumerate(slc_zips_list):
    esd_outpaths.append(os.path.join(esds_path, f'{slc_zips_dates[i]}.dim'))
    esd_cmd = f'{GPT_PATH} Enhanced-Spectral-Diversity -Ssource={geocode_outpaths[i]} -PcohThreshold={COHERENCE_THRESHOLD} -PesdEstimator={ESD_ESTIMATOR} -PfineWinAccAzimuth={WIN_ACC_AZI} -PfineWinAccRange={WIN_ACC_RANGE} -PfineWinHeightStr={WIN_HEIGHT} -PfineWinOversampling={WIN_OVERSAMPLING} -PfineWinWidthStr={WIN_WIDTH} -PintegrationMethod="{INTEGRATION_METHOD}" -PnumBlocksPerOverlap={NUM_BLOCKS_PER_OVERLAP} -PoverallAzimuthShift={OVERALL_AZI_SHIFT} PoverallRangeShift={OVERALL_RANGE_SHIFT} -PweightFunc="{WEIGHT_FUNCTION}" -PxCorrThreshold={XCOR_THRESHOLD} -t {esd_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(esd_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(geocodes_path)

# 4b. Generate Ifg
- may leave out the coherence and elevation bands from here. Could be added in individual steps later? Going to test first though.

In [ ]:
COHERENCE_AZI_WIN = 10              
COHERENCE_RANGE_WIN = 10
OUTPUT_ELEVATION = True                 # boolean
OUTPUT_FLAT_EARTH_PHASE = False         # boolean
OUTPUT_LAT_LON = False                  # boolean
OUTPUT_TOPO_PHASE = False               # boolean
SQUARE_PIXEL = True                     # boolean

# these are for flat earth phase estimation
FEP_NUM_POINTS = 501                    # 301, 401, 501, 601, 701, 801, 901, 1001
FEP_POLY_DEGREE = 5                     # 1, 2, 3, 4, 5, 6, 7, 8

SUBTRACT_FEP = True                     # boolean
SUBTRACT_TOPO_PHASE = False             # boolean   
TILE_EXTENSION_PERCENT = 100

ifgs_path = os.path.join(if_dir, '05_interferograms')
os.makedirs(ifgs_path, exist_ok=True, mode=0o777)

ifg_outpaths = []
for i, file in enumerate(slc_zips_list):
    ifg_outpaths.append(os.path.join(ifgs_path, f'{slc_zips_dates[i]}.dim'))

    ifg_cmd = f'{GPT_PATH} Interferogram -SsourceProduct={esd_outpaths[i]} -PcohWinAz={COHERENCE_AZI_WIN} -PcohWinRg={COHERENCE_RANGE_WIN} -PdemName={DEM_NAME} -PorbitDegree={ORBIT_DEGREE} -PoutputElevation={OUTPUT_ELEVATION} -PoutputFlatEarthPhase={OUTPUT_FLAT_EARTH_PHASE} -PoutputLatLon={OUTPUT_LAT_LON} -PoutputTopoPhase={OUTPUT_TOPO_PHASE} -PsquarePixel={SQUARE_PIXEL} -PsrpNumberPoints={FEP_NUM_POINTS} -PsrpPolynomialDegree={FEP_POLY_DEGREE} -PsubtractFlatEarthPhase={SUBTRACT_FEP} -PsubtractTopographicPhase={SUBTRACT_TOPO_PHASE} -PtileExtensionsPercent={TILE_EXTENSION_PERCENT} -t {ifg_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(ifg_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(esds_path)

# 5. TOPSAR-Deburst

In [ ]:
debursts_path = os.path.join(if_dir, '06_deburst')
os.makedirs(debursts_path, exist_ok=True, mode=0o777)

deburst_outpaths = []
for i, file in enumerate(slc_zips_list):
    deburst_outpaths.append(os.path.join(debursts_path, f'{slc_zips_dates[i]}.dim'))

    deburst_cmd = f'{GPT_PATH} TOPSAR-Deburst -Ssource={ifg_outpaths[i]} -PselectedPolarisations={POLARISATIONS} -t {deburst_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(deburst_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(ifgs_path)

# 6. Multilook

In [ ]:
SQUARE_PIXEL = False    # boolean
AZI_LOOKS = '1'         # sentinel-1 is 22m in azimuth
RANGE_LOOKS = '5'       # sentinel-1 is between 2.7 and 3.5m in range, depending on topography
SOURCE_BANDS = ''       # reference band used to multilook

mlooks_path = os.path.join(if_dir, '07_multilook')
os.makedirs(mlooks_path, exist_ok=True, mode=0o777)

mlook_outpaths = []
for i, file in enumerate(slc_zips_list):
    mlook_outpaths.append(os.path.join(mlooks_path, f'{slc_zips_dates[i]}.dim'))

    mlook_cmd = f'{GPT_PATH} Multilook -Ssource={deburst_outpaths[i]} -PgrSquarePixel={SQUARE_PIXEL} -PnAzLooks={AZI_LOOKS} -PnRgLooks={RANGE_LOOKS} -t {mlook_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(mlook_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(debursts_path)

# 7. Topo-phase removal
- may not be needed since it can be done within ifg generation? Should probably test on a single ifg to compare differences and identify ideal workflow

In [ ]:
OUTPUT_ELEVATION = True                 # boolean
OUTPUT_LAT_LON = False                  # boolean
OUTPUT_TOPO_PHASE = False               # boolean
TILE_EXTENSION_PERCENT = 100

topos_path = os.path.join(if_dir, '08_topophaseremoval')
os.makedirs(topos_path, exist_ok=True, mode=0o777)

topo_outpaths = []
for i, file in enumerate(slc_zips_list):
    topo_outpaths.append(os.path.join(topos_path, f'{slc_zips_dates[i]}.dim'))

    topo_cmd = f'{GPT_PATH} TopoPhaseRemoval -Ssource={mlook_outpaths[i]} -PgrSquarePixel={SQUARE_PIXEL} -PnAzLooks={AZI_LOOKS} -PnRgLooks={RANGE_LOOKS} -t {topo_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(topo_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(mlooks_path)

# 8. Goldstein Phase Filtering

In [ ]:
ALPHA = 1.0                     # double, (0, 1], default 1.0
FFT_SIZE = 64                   # 32, 64, 128, 256
COHERENCE_MASK = False          # boolean, masks out low coherence pixels during filtering
WINDOW_SIZE = 3                 # 3, 5, 7

filters_path = os.path.join(if_dir, '09_goldsteinfiltering')
os.makedirs(filters_path, exist_ok=True, mode=0o777)

filter_outpaths = []
for i, file in enumerate(slc_zips_list):
    filter_outpaths.append(os.path.join(filters_path, f'{slc_zips_dates[i]}.dim'))

    filter_cmd = f'{GPT_PATH} GoldsteinPhaseFiltering -Ssource={topo_outpaths[i]} -Palpha={ALPHA} -PcoherenceThreshold={COHERENCE_THRESHOLD} -PFFTSizeString={FFT_SIZE} -PuseCoherenceMask={COHERENCE_MASK} -PwindowSizeString={WINDOW_SIZE} -t {filter_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(filter_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(topos_path)

# 9a. Merge (if multiple subswaths)

In [ ]:
merges_path = os.path.join(if_dir, '10_merged')
os.makedirs(merges_path, exist_ok=True, mode=0o777)

merge_outpaths = []
for i, file in enumerate(slc_zips_list):
    merge_outpaths.append(os.path.join(merges_path, f'{slc_zips_dates[i]}.dim'))

    merge_cmd = f'{GPT_PATH} GoldsteinPhaseFiltering -SmasterProduct={filter_outpaths[i]} -t {merge_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(merge_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(filters_path)

# 9b. Subset and extract wrapped Ifg

In [ ]:
COPY_METADATA = True            # boolean
FULL_SWATH = False              # boolean
GEO_REGION =                    # wkt region
REFERENCE_BAND = 'VV'           # band used as reference for coordinates. CHECK THE AVAILABLE BANDS IN SNAP BEFORE PROCESSING
SOURCE_BANDS = 'VV'
X_SUB_SAMPLING = 1              # int, step size for subsampling in horizontal direction
Y_SUB_SAMPLING = 1              # int, step size for subsampling in vertical direction
# TIE_POINT_GRIDS =               # HONESTLY NO CLUE. LOOK INTO THIS MORE IN GUI

subsets_path = os.path.join(if_dir, '11_subset')
os.makedirs(subsets_path, exist_ok=True, mode=0o777)

subset_outpaths = []
for i, file in enumerate(slc_zips_list):
    subset_outpaths.append(os.path.join(subsets_path, f'{slc_zips_dates[i]}.dim'))

    subset_cmd = f'{GPT_PATH} GoldsteinPhaseFiltering -Ssource={merge_outpaths[i]} -t {subset_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(subset_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(merges_path)

# 10. Add and extract elevation band only (needed for attribute metadata)
- if elevation included in ifg generation, just extract and export?
- may need to create custom .xml within gui first to be able to extract just elevation band

# 11. Add and extract coherence band
- if coherence included in ifg generation, just extract and export?
- may need to create custom .xml within gui first to be able to extract just coherence band

# 12. SNAPHU Export

In [ ]:
help_cmd('SnaphuExport')

# 13. SNAPHU unwrap
- use subprocess
- CLI example: snaphu -f snaphu.conf Phase_ifg_VV_28Mar2010_02May2010.snaphu.img 5191

# 14. SNAPHU Import => Unwrapped Ifg

# 15. Terrain Correction for 8b, 9, 10, and 13

In [ ]:
ALIGN_STANDARD_GRID = False
RADIOMETRIC_NORMALIZATION = True
AUX_FILE = 'Product Auxiliary File'   # str, options include 'Latest Auxiliary File', 'Product Auxiliary File', 'External Auxiliary File'
DEM_NAME = 'Copernicus 30m Global DEM'
DEM_RESAMPLE_METHOD = 'DELAUNAY_INTERPOLATION'   # str, options include 'NEAREST_NEIGHBOUR', 'BILINEAR_INTERPOLATION', 'CUBIC_CONVOLUTION', 'BISINC_5_POINT_INTERPOLATION', 'BISINC_11_POINT_INTERPOLATION', 'BISINC_21_POINT_INTERPOLATION', 'BICUBIC_INTERPOLATION', 'DELAUNAY_INTERPOLATION'
IMG_RESAMPLE_METHOD = 'CUBIC_CONVOLUTION' # str, options include 'NEAREST_NEIGHBOUR', 'BILINEAR_INTERPOLATION', 'CUBIC_CONVOLUTION', 'BISINC_5_POINT_INTERPOLATION', 'BISINC_11_POINT_INTERPOLATION', 'BISINC_21_POINT_INTERPOLATION', 'BICUBIC_INTERPOLATION'
NODATA_SEA = False
OUTPUT_COMPLEX = False
PIXEL_SPACING_METERS = 20.0   # double
SAVE_DEM = False
SAVE_ANGLE_ELLIPSOID = False
SAVE_LAT_LON = False
SAVE_LAYOVER_SHADOW = True
SAVE_LOCAL_ANGLE = False
SAVE_PROJ_LOCAL_ANGLE = True
SAVE_SOURCE_BAND = True
SOURCE_BANDS = 'Gamma0_VV,Gamma0_VH'
GRID_ORIGIN_X = 0   # double
GRID_ORIGIN_Y = 0   # double

# only needed if you you are using external dem
# uncomment these and adjust your gpt command below accordingly
#  EXTERNAL_AUX = 
# EXTERNAL_DEMFILE =
# EXTERNAL_DEMFILE_NODATA =
# ETERNAL_DEM_EGM = False

tc_outpaths = []
for i, file in enumerate(slc_zips_list):
    tc_outpaths.append(os.path.join(grdpaths[8], f'{slc_zips_dates[i]}.dim'))

    tc_cmd = f'{GPT_PATH} Terrain-Correction -Ssource={tf_outpaths[i]} -PalignToStandardGrid={ALIGN_STANDARD_GRID} -PapplyRadiometricNormalization={RADIOMETRIC_NORMALIZATION} -PauxFile="{AUX_FILE}" -PdemName="{DEM_NAME}" -PdemResamplingMethod="{DEM_RESAMPLE_METHOD}" -PimgResamplingMethod="{IMG_RESAMPLE_METHOD}" -PnodataValueAtSea={NODATA_SEA} -PoutputComplex={OUTPUT_COMPLEX} -PpixelSpacingInMeter={PIXEL_SPACING_METERS} -PsaveDEM={SAVE_DEM} -PsaveIncidenceAngleFromEllipsoid={SAVE_ANGLE_ELLIPSOID} -PsaveLatLon={SAVE_LAT_LON} -PsaveLayoverShadowMask={SAVE_LAYOVER_SHADOW} -PsaveLocalIncidenceAngle={SAVE_LOCAL_ANGLE} -PsaveProjectedLocalIncidenceAngle={SAVE_PROJ_LOCAL_ANGLE} -PsaveSelectedSourceBand={SAVE_SOURCE_BAND} -PsourceBands={SOURCE_BANDS} -PstandardGridOriginX={GRID_ORIGIN_X} -PstandardGridOriginY={GRID_ORIGIN_Y} -t {tc_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(tc_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

#     os.remove(tf_outpaths[i]) # removes the just used source product
#     shutil.rmtree(tf_outpaths[i][:-4]+'.data')

# shutil.rmtree(grdpaths[7])

# 16. Final subset

In [ ]:
COPY_METADATA = True
FULL_SWATH = False
REFERNCE_BAND = 'Beta0_VV'

subset_outpaths = []
for i, file in enumerate(slc_zips_list):
    subset_outpaths.append(os.path.join(grdpaths[5], f'{slc_zips_dates[i]}.dim'))

    subset_cmd = f'{GPT_PATH} Subset -Ssource={mlook_outpaths[i]} -PcopyMetadata={COPY_METADATA} -PfullSwath={FULL_SWATH} -PgeoRegion="{aoi}" -PreferenceBand={REFERNCE_BAND} -t {subset_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(subset_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

    os.remove(mlook_outpaths[i]) # removes the just used source product
    shutil.rmtree(mlook_outpaths[i][:-4]+'.data')

shutil.rmtree(grdpaths[4])